# IEEE-CIS Fraud Detection - data analisys

In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, ensemble, metrics, linear_model, svm
#, naive_bayes, metrics, svm, decomposition
import xgboost 
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

warnings.simplefilter('ignore')
__location__ = sys.path[0] 
output_path = f'{__location__}/Input/ieee-fraud-detection/sample_submission.csv'

Using TensorFlow backend.


In [2]:
#train_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_transaction.csv')
#test_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_identity.csv')
#test_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_transaction.csv')

In [3]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()

non_num_cols = [train_transaction.columns[n] for n,i in enumerate(train_transaction.dtypes)
                if i not in ('int64','float64')]

for i in non_num_cols:
    #Filling missing string values with most common value
    train_transaction['mod_'+i] = encoder.fit_transform(train_transaction[i].fillna(train_transaction[i].mode()[0]))
    #test_transaction['mod_'+i] = encoder.fit_transform(test_transaction[i].fillna(test_transaction[i].mode()[0])

In [4]:
col_list = [x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]

x_train, y_train, x_valid, y_valid = model_selection.train_test_split(train_transaction[col_list].fillna(0)
                                                                      , train_transaction['isFraud']
                                                                      , test_size=0.9
                                                                     )
scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(x_train)
x_train = scaler.transform(x_train)
y_train = scaler.transform(y_train)

In [6]:
from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=0)
#x_train, x_valid = ros.fit_resample(x_train, x_valid)
x_train, x_valid = SMOTE().fit_resample(x_train, x_valid)

In [14]:
RF = ensemble.RandomForestClassifier(n_jobs=-1) #97.25
RF.fit(x_train, x_valid)
predictions = RF.predict(y_train)
accuracy = metrics.accuracy_score(predictions, y_valid)
print (f"RF, xx={0}: ", accuracy)


LR = linear_model.LogisticRegression()#penalty = 'l2', C = 100) -- 82.66

LR.fit(x_train, x_valid)
predictions = LR.predict(y_train)
accuracy = metrics.accuracy_score(predictions, y_valid)
print (f"LR, c_params_range={100}: ", accuracy)

RF, xx=0:  0.9725147981320298
LR, c_params_range=100:  0.7994246320693301


In [17]:
XGb = xgboost.XGBClassifier(silent=False, 
                            #scale_pos_weight=1,
                            learning_rate=0.01,  
                            #colsample_bytree = 0.4,
                            #subsample = 0.8,
                            #objective='binary:logistic', 
                            n_estimators=100#, 
                            #reg_alpha = 0.3,
                            #max_depth=4, 
                            #gamma=10
                           ) 
#() 96

XGb.fit(x_train, x_valid)
predictions = XGb.predict(y_train)
accuracy = metrics.accuracy_score(predictions, y_valid)
print (f"XGb: ", accuracy)

XGb:  0.8819554983574356
